In [1]:
%cd ..
#import site
#site.addsitedir('C:/Users/Admin/Documents/GitHub/blokus/')  # Replace with the actual path to your project folder
from src import bot
import src.objects as o
from src.objects import Player, Piece
from pygame import display, Surface, font, image, surfarray, Rect
import pygame
import src.events as e
import src.views as v
from os.path import join
import src.controllers as c
import numpy as np
import src.reward as r

c:\Users\Admin\Documents\GitHub\blokus


pygame 2.5.2 (SDL 2.28.3, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import numpy as np

In [3]:
from src import bot
board = o.LinkedGrid(20,20, 20)
players = []
colors = ["r", "g", "b", "y"]
for p in range(4):
    players.append(Player(colors[p]))
p_i = 0

In [4]:
run = True
weights = [1,1,10]
turn = 0

player = players[p_i]


if False not in [player.isDone for player in players]: # when all players are done, game stops.
    run = False
    

if not player.isDone:
    matrix = bot.convert_matrix_to_nparray(board.matrix)
    possible_places, possible_plays_indices, possible_pieces = bot.get_available_actions(matrix, player)
    possible_places = [None if lst is None else tuple(lst) for lst in possible_places]
    if len(possible_plays_indices) == 0:
        player.isDone = True
    else:
        # Random play right now.
        best = 0
        choice = 0
        best_reward = None

        for i in range(len(possible_places)):
            reward = bot.estimate_rewards(matrix, possible_places[i], bot.convert_color_to_number(player.c))
            current_sum = sum([x * y for x, y in zip(weights, reward)])
            if current_sum > best:
                choice = i
                best = current_sum
                best_reward = reward

        # rand_index = bot.random_index(possible_plays_indices)
        pkey, pindex = possible_plays_indices[choice]
        absolute_coords = possible_places[choice]
        piece_played = possible_pieces[choice]
        #print(choice)
        bot.play_coordinates(pkey, pindex, absolute_coords, piece_played, player, board)
        if len(player.pieces) == 0:
            player.isDone = True
        print(bot.convert_matrix_to_nparray(board.matrix))

p_i+=1
if p_i>3: 
    p_i=0

[[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [5]:
def choose_move_depth(matrix, possible_places, color, depth = 3, num_choices = 5):
    best = []
    weights = [10,10,1]
    choice = []
    old_square = []
    corners = {}
    for c in range(1,5):
        corners[c]= set(bot.get_playable_conditions(matrix,color, only_corners= True))  
    for i in range(len(possible_places)):
        reward, new_square = r.estimate_rewards(matrix, possible_places[i], color, corners, old_square)
        current_sum = sum([x * y for x, y in zip(weights, reward)])

        if len(choice) < num_choices:
             best.append((current_sum,new_square))
             choice.append(i)
        else:
            rewards = [r for r,s in best]
            min_index, min_reward = np.argmin(rewards), np.min(rewards)
            if current_sum > min_reward:
                 best.pop(min_index)
                 best.append((current_sum, new_square))
                 choice.pop(min_index)
                 choice.append(i)

    return choice

In [6]:
matrix = bot.convert_matrix_to_nparray(board.matrix)
possible_places, possible_plays_indices, possible_pieces = bot.get_available_actions(matrix, player)
possible_places = [None if lst is None else tuple(lst) for lst in possible_places]

In [7]:
for i in range(10):
    choose_move_depth(matrix, possible_places, bot.convert_color_to_number(player.c) )

In [8]:
colors = [1,2,3,4]
corners = {}
for c in colors:
    corners[c]= set(bot.get_playable_conditions(matrix,c, only_corners= True) )
corners

{1: {(0, 3), (2, 0), (3, 1), (3, 3)}, 2: {(0, 19)}, 3: {(0, 19)}, 4: {(0, 19)}}

In [9]:
r.get_number_of_blocked_corners(corners,([2,0], None, [2,19]),3)

1

In [10]:
def get_number_of_corners(matrix, absolute_coords, color, corners):
    rows, cols = len(matrix), len(matrix[0])
    added_corners = 0
    removed_corners = 0
    diagonals = [(-1, -1), (-1, 1), (1, -1), (1, 1)]
    laterals = [(0, -1), (0, 1), (-1, 0), (1, 0)]
    
    for coord in absolute_coords:
        if coord is not None:
            i,j = coord
            for x, y in diagonals:
                    new_i, new_j = i + x, j + y
                    if 0 <= new_i < rows and 0 <= new_j < cols:
                        if matrix[new_i][new_j] == 0:
                            append = True
                            for a, b in laterals:    
                            # Check for valid corners
                                if 0 <= new_i+a < rows and 0 <= new_j+b < cols:
                                    if (matrix[new_i+a][new_j+b] == color) or ([new_i+a,new_j+b] in absolute_coords):
                                        append = False
                                        break
                            #Check if corner already exists
                            if (new_i,new_j) in corners[color]:
                                 append = False

                            if append:
                                added_corners += 1

    for coord in corners[color]:
        i,j = coord
        remove = False
        #Check if a corner has been occupied
        if [i,j] in absolute_coords:
            remove = True
        
            
        for a, b in laterals:
        # Check if squares surrounding corner have been occupied
                if [i+a,j+b] in absolute_coords:
                    remove = True
                    break
        if remove:
            removed_corners += 1


    return added_corners, removed_corners

In [11]:
absolute_coords = ([3,16],[4,16],[4,17])
for i in range(10000):
    get_number_of_corners(matrix,absolute_coords, 2, corners)

In [12]:
corners[2]

{(0, 19)}

In [13]:
matrix = bot.convert_matrix_to_nparray(board.matrix)
for i in range(10000):
    bot.get_number_of_blocked_corners(matrix, matrix, 2)

In [14]:
[3,3] in absolute_coords

False

In [15]:
bot.convert_matrix_to_nparray(board.matrix)

array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0,

In [16]:
a,b,c = bot.get_available_actions(bot.convert_matrix_to_nparray(board.matrix),player)

In [19]:
board = o.LinkedGrid(20,20, 20)
players = []
colors = ["r", "g", "b", "y"]
weights = [[10,10,1],[10,10,1],[10,10,1],[3,2,1]]
for p in range(4):
    players.append(Player(colors[p]))

#Initialize player strategies for comparison
for p in range(4):
    players[p].weights = weights[p]


In [22]:
piece_opening = []
for key in players[0].pieces.keys():
    for i,piece in enumerate(players[0].pieces[key]):
        piece_opening.append(piece.m)

In [24]:
matrix = bot.convert_matrix_to_nparray(board.matrix)
possible_places, possible_plays_indices, possible_pieces = bot.get_available_actions(matrix, players[0])

In [35]:
piece_opening[0].tolist()

[[1]]

In [38]:
possible_pieces.index(piece_opening[3].tolist())

24

In [ ]:
a = (np.array([0], dtype = int),np.array([0], dtype = int))

In [ ]:
import numpy as np

In [ ]:
best = [3,5,6,7]
min_index, min_value = np.argmin(best), np.min(best)





In [ ]:
min_value

3